In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
home_dir = "../"

import os
import pandas as pd

In [2]:
snps_with_alfa_mane_df = pd.read_csv(home_dir+"data/temp_popufreq_data/snps_with_alfa_mane_raw.tsv", sep="\t")
print(snps_with_alfa_mane_df.columns)
print(snps_with_alfa_mane_df.shape)
print(snps_with_alfa_mane_df["snp_id"].unique().shape[0])

/tmp/ipykernel_3805907/3102721948.py:1: DtypeWarning: Columns (47) have mixed types. Specify dtype option on import or set low_memory=False.
  snps_with_alfa_mane_df = pd.read_csv(home_dir+"data/temp_popufreq_data/snps_with_alfa_mane_raw.tsv", sep="\t")


Index(['snp_id', 'acc', 'chrpos', 'spdi', 'tax_id', 'snp_class', 'create_date',
       'update_date', 'clinical_significance', 'fxn_class', 'validated',
       'genes', 'mafs', 'variations', 'CHROM', 'POS', 'ID', 'REF', 'ALT',
       'QUAL', 'FILTER', 'INFO', 'FORMAT', 'SAMN10492695', 'SAMN10492696',
       'SAMN10492697', 'SAMN10492698', 'SAMN10492699', 'SAMN10492700',
       'SAMN10492701', 'SAMN10492702', 'SAMN11605645', 'SAMN10492703',
       'SAMN10492704', 'SAMN10492705', 'gene_symbol', 'gene_id',
       '#NCBI_GeneID', 'Ensembl_Gene', 'HGNC_ID', 'symbol', 'name',
       'RefSeq_nuc', 'RefSeq_prot', 'Ensembl_nuc', 'Ensembl_prot',
       'MANE_status', 'GRCh38_chr', 'chr_start', 'chr_end', 'chr_strand',
       'mane_protein_variants'],
      dtype='object')
(1668243, 52)
1656512


In [3]:
def parse_spdi(row):
    # print(row)
    spdi_variants = row.spdi.split(",") # NC_000011.10:4390048:G:A,NC_000011.10:4390048:G:C,NC_000011.10:4390048:G:T
    variants = []
    for spdi_variant in spdi_variants:
        variant = spdi_variant.split(":")[2:]
        variant = ">".join(variant)
        variants.append(variant)
        
    return ",".join(variants)

snps_with_alfa_mane_df["spdi_genomic_variants"] = snps_with_alfa_mane_df.apply(parse_spdi, axis=1)
print(snps_with_alfa_mane_df.shape)
print(snps_with_alfa_mane_df["snp_id"].unique().shape[0])
snps_with_alfa_mane_df["spdi_genomic_variants"]

(1668243, 53)
1656512


0                  A>G
1                  T>A
2                  A>G
3                  T>C
4                  C>T
              ...     
1668238    C>A,C>G,C>T
1668239    A>C,A>G,A>T
1668240    T>A,T>C,T>G
1668241            A>G
1668242            T>C
Name: spdi_genomic_variants, Length: 1668243, dtype: object

In [4]:
def is_sameNum_ALFAGenomic_and_MANEProt_variants(row):
    alfa_genomic_variants = row.ALT.split(",")
    mane_protein_variants = row.mane_protein_variants.split(",")
    if len(alfa_genomic_variants) == len(mane_protein_variants): return True
    return False

snps_with_alfa_mane_sameNum_ALFAGenomic_and_MANEProt_variants_df = snps_with_alfa_mane_df[snps_with_alfa_mane_df.apply(is_sameNum_ALFAGenomic_and_MANEProt_variants, axis=1)]
snps_with_alfa_mane_sameNum_ALFAGenomic_and_MANEProt_variants_df.shape

(1383922, 53)

In [5]:
snps_with_alfa_mane_sameNum_ALFAGenomic_and_MANEProt_variants_df["genomic_to_prot_variants_mapping_source"] = "ALFA"

/tmp/ipykernel_3805907/4222152510.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  snps_with_alfa_mane_sameNum_ALFAGenomic_and_MANEProt_variants_df["genomic_to_prot_variants_mapping_source"] = "ALFA"


In [6]:
snps_with_alfa_mane_notSameNum_ALFAGenomic_and_MANEProt_variants_df = snps_with_alfa_mane_df[~snps_with_alfa_mane_df.apply(is_sameNum_ALFAGenomic_and_MANEProt_variants, axis=1)]
snps_with_alfa_mane_notSameNum_ALFAGenomic_and_MANEProt_variants_df.shape

(284321, 53)

In [7]:
def is_sameNum_SPDIGenomic_and_MANEProt_variants(row):
    spdi_genomic_variants = row.spdi_genomic_variants.split(",")
    mane_protein_variants = row.mane_protein_variants.split(",")
    if len(spdi_genomic_variants) == len(mane_protein_variants): return True
    return False

snps_with_alfa_mane_sameNum_SPDIGenomic_and_MANEProt_variants_df = snps_with_alfa_mane_notSameNum_ALFAGenomic_and_MANEProt_variants_df[snps_with_alfa_mane_notSameNum_ALFAGenomic_and_MANEProt_variants_df.apply(is_sameNum_SPDIGenomic_and_MANEProt_variants, axis=1)]
snps_with_alfa_mane_sameNum_SPDIGenomic_and_MANEProt_variants_df.shape

(221830, 53)

In [8]:
snps_with_alfa_mane_sameNum_SPDIGenomic_and_MANEProt_variants_df["genomic_to_prot_variants_mapping_source"] = "SPDI-ALFA"

/tmp/ipykernel_3805907/3171419286.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  snps_with_alfa_mane_sameNum_SPDIGenomic_and_MANEProt_variants_df["genomic_to_prot_variants_mapping_source"] = "SPDI-ALFA"


In [9]:
snps_with_alfa_mane_notSameNum_SPDIGenomic_and_MANEProt_variants_df = snps_with_alfa_mane_notSameNum_ALFAGenomic_and_MANEProt_variants_df[~snps_with_alfa_mane_notSameNum_ALFAGenomic_and_MANEProt_variants_df.apply(is_sameNum_SPDIGenomic_and_MANEProt_variants, axis=1)]
snps_with_alfa_mane_notSameNum_SPDIGenomic_and_MANEProt_variants_df.shape

(62491, 53)

In [10]:
snps_with_alfa_mane_notSameNum_SPDIGenomic_and_MANEProt_variants_df["genomic_to_prot_variants_mapping_source"] = "uncertain"

/tmp/ipykernel_3805907/2285177060.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  snps_with_alfa_mane_notSameNum_SPDIGenomic_and_MANEProt_variants_df["genomic_to_prot_variants_mapping_source"] = "uncertain"


In [11]:
snps_with_alfa_mane_genomicToProtMapping_df = pd.concat([snps_with_alfa_mane_sameNum_ALFAGenomic_and_MANEProt_variants_df, snps_with_alfa_mane_sameNum_SPDIGenomic_and_MANEProt_variants_df, snps_with_alfa_mane_notSameNum_SPDIGenomic_and_MANEProt_variants_df])
print(snps_with_alfa_mane_genomicToProtMapping_df.shape)
snps_with_alfa_mane_genomicToProtMapping_df["genomic_to_prot_variants_mapping_source"].value_counts()

(1668243, 54)


ALFA         1383922
SPDI-ALFA     221830
uncertain      62491
Name: genomic_to_prot_variants_mapping_source, dtype: int64

In [12]:
def map_genomicToProt_variants(row):
    # print(row)
    if row.genomic_to_prot_variants_mapping_source == "uncertain":
        prot_variants = None
    elif row.genomic_to_prot_variants_mapping_source == "ALFA":
        prot_variants = row.mane_protein_variants
    elif row.genomic_to_prot_variants_mapping_source == "SPDI-ALFA":
        alfa_alt_alleles = row.ALT.split(",")
        spdi_alt_alleles_dict = {x.split(">")[1]:i for i, x in enumerate(row.spdi_genomic_variants.split(","))}
        mane_protein_variants = row.mane_protein_variants.split(",")
        prot_variants = []
        for alfa_alt_allele in alfa_alt_alleles:
            idx = spdi_alt_alleles_dict[alfa_alt_allele]
            prot_variants.append(mane_protein_variants[idx])
        prot_variants = ",".join(prot_variants)
    return prot_variants

snps_with_alfa_mane_genomicToProtMapping_df["genomic_to_mapped_prot_variants"] = snps_with_alfa_mane_genomicToProtMapping_df.apply(map_genomicToProt_variants, axis=1)
snps_with_alfa_mane_genomicToProtMapping_df.shape

(1668243, 55)

In [13]:
print(snps_with_alfa_mane_genomicToProtMapping_df["genomic_to_prot_variants_mapping_source"].value_counts())
snps_with_alfa_mane_genomicToProtMapping_df.columns

ALFA         1383922
SPDI-ALFA     221830
uncertain      62491
Name: genomic_to_prot_variants_mapping_source, dtype: int64


Index(['snp_id', 'acc', 'chrpos', 'spdi', 'tax_id', 'snp_class', 'create_date',
       'update_date', 'clinical_significance', 'fxn_class', 'validated',
       'genes', 'mafs', 'variations', 'CHROM', 'POS', 'ID', 'REF', 'ALT',
       'QUAL', 'FILTER', 'INFO', 'FORMAT', 'SAMN10492695', 'SAMN10492696',
       'SAMN10492697', 'SAMN10492698', 'SAMN10492699', 'SAMN10492700',
       'SAMN10492701', 'SAMN10492702', 'SAMN11605645', 'SAMN10492703',
       'SAMN10492704', 'SAMN10492705', 'gene_symbol', 'gene_id',
       '#NCBI_GeneID', 'Ensembl_Gene', 'HGNC_ID', 'symbol', 'name',
       'RefSeq_nuc', 'RefSeq_prot', 'Ensembl_nuc', 'Ensembl_prot',
       'MANE_status', 'GRCh38_chr', 'chr_start', 'chr_end', 'chr_strand',
       'mane_protein_variants', 'spdi_genomic_variants',
       'genomic_to_prot_variants_mapping_source',
       'genomic_to_mapped_prot_variants'],
      dtype='object')

In [14]:
snps_with_alfa_mane_genomicToProtMapping_df.to_csv(home_dir+"data/temp_popufreq_data/snps_with_alfa_mane_genomicToProtMapping.tsv", index=False, sep="\t")

In [15]:
cols_dict = {"snp_id": "snp_id", "spdi":"spdi", "genes": "snp_raw_genes", "gene_symbol": "gene_symbol", "gene_id": "gene_id", 
             
             "RefSeq_nuc": "mane_refseq_nuc", "RefSeq_prot": "mane_refseq_prot", "MANE_status": "mane_status", 
             "GRCh38_chr": "mane_grch38_chr_no", "chr_start": "mane_chr_start", "chr_end": "mane_chr_end", "chr_strand": "mane_chr_strand", 
             "mane_protein_variants": "mane_protein_variants",

             'spdi_genomic_variants': 'spdi_genomic_variants',
             'genomic_to_prot_variants_mapping_source': "genomic_to_prot_variants_mapping_source",
             'genomic_to_mapped_prot_variants': "genomic_to_mapped_prot_variants",

             "CHROM": "alfa_chrom_acc_version", "POS": "alfa_1indexed_chrom_pos", "REF": "alfa_ref_allele", "ALT": "alfa_alt_alleles", "SAMN10492705": "alfa_popu_counts"
             }

snps_with_alfa_mane_short_df = snps_with_alfa_mane_genomicToProtMapping_df[cols_dict.keys()].copy()
snps_with_alfa_mane_short_df.rename(columns=cols_dict, inplace=True)
snps_with_alfa_mane_short_df.to_csv(home_dir+"data/temp_popufreq_data/snps_with_alfa_mane_genomicToProtMapping_short.tsv", index=False, sep="\t")

In [16]:
uncertain_snps_df = snps_with_alfa_mane_short_df[snps_with_alfa_mane_short_df["genomic_to_prot_variants_mapping_source"]=="uncertain"]
uncertain_snps_df.to_csv(home_dir+"data/temp_popufreq_data/snps_with_alfa_mane_genomicToProtMapping_short_uncertain_snps.tsv", index=False, sep="\t")

In [17]:
snps_with_alfa_mane_short_SPDI_ALFA_df = snps_with_alfa_mane_short_df[snps_with_alfa_mane_short_df["genomic_to_prot_variants_mapping_source"]=="SPDI-ALFA"]
snps_with_alfa_mane_short_SPDI_ALFA_df.to_csv(home_dir+"data/temp_popufreq_data/snps_with_alfa_mane_genomicToProtMapping_short_SPDI_ALFA.tsv", index=False, sep="\t")

In [18]:
snps_with_alfa_mane_short_df = pd.read_csv(home_dir+"data/temp_popufreq_data/snps_with_alfa_mane_genomicToProtMapping_short.tsv", sep="\t")
snps_with_alfa_mane_short_df.head(50)

/tmp/ipykernel_3805907/3403666447.py:1: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  snps_with_alfa_mane_short_df = pd.read_csv(home_dir+"data/temp_popufreq_data/snps_with_alfa_mane_genomicToProtMapping_short.tsv", sep="\t")


,snp_id,spdi,snp_raw_genes,gene_symbol,gene_id,mane_refseq_nuc,mane_refseq_prot,mane_status,mane_grch38_chr_no,mane_chr_start,...,mane_chr_strand,mane_protein_variants,spdi_genomic_variants,genomic_to_prot_variants_mapping_source,genomic_to_mapped_prot_variants,alfa_chrom_acc_version,alfa_1indexed_chrom_pos,alfa_ref_allele,alfa_alt_alleles,alfa_popu_counts
0,rs1491003598,NC_000001.11:205269217:A:G,TMCC2:9911,TMCC2,9911,NM_014858.4,NP_055673.2,MANE Select,1,205227946,...,+,NP_055673.2:p.Lys339Arg,A>G,ALFA,NP_055673.2:p.Lys339Arg,NC_000001.11,205269218,A,G,8988:1
1,rs1436854664,NC_000001.11:205271827:T:A,TMCC2:9911,TMCC2,9911,NM_014858.4,NP_055673.2,MANE Select,1,205227946,...,+,NP_055673.2:p.Cys612Ser,T>A,ALFA,NP_055673.2:p.Cys612Ser,NC_000001.11,205271828,T,A,14050:1
2,rs1431352460,NC_000001.11:205241636:A:G,TMCC2:9911,TMCC2,9911,NM_014858.4,NP_055673.2,MANE Select,1,205227946,...,+,NP_055673.2:p.Met114Val,A>G,ALFA,NP_055673.2:p.Met114Val,NC_000001.11,205241637,A,G,32062:1
3,rs1429019865,NC_000001.11:205241751:T:C,TMCC2:9911,TMCC2,9911,NM_014858.4,NP_055673.2,MANE Select,1,205227946,...,+,NP_055673.2:p.Leu152Pro,T>C,ALFA,NP_055673.2:p.Leu152Pro,NC_000001.11,205241752,T,C,4470:1
4,rs1423575191,NC_000001.11:205269046:C:T,TMCC2:9911,TMCC2,9911,NM_014858.4,NP_055673.2,MANE Select,1,205227946,...,+,NP_055673.2:p.Pro282Leu,C>T,ALFA,NP_055673.2:p.Pro282Leu,NC_000001.11,205269047,C,T,21382:2
5,rs1415408852,NC_000001.11:205241615:C:A,TMCC2:9911,TMCC2,9911,NM_014858.4,NP_055673.2,MANE Select,1,205227946,...,+,NP_055673.2:p.Gln107Lys,C>A,ALFA,NP_055673.2:p.Gln107Lys,NC_000001.11,205241616,C,A,21382:1
6,rs1410312003,NC_000001.11:205241510:C:G,TMCC2:9911,TMCC2,9911,NM_014858.4,NP_055673.2,MANE Select,1,205227946,...,+,NP_055673.2:p.Gln72Glu,C>G,ALFA,NP_055673.2:p.Gln72Glu,NC_000001.11,205241511,C,G,14050:1
7,rs1401358469,NC_000001.11:205271241:C:T,TMCC2:9911,TMCC2,9911,NM_014858.4,NP_055673.2,MANE Select,1,205227946,...,+,NP_055673.2:p.Ala602Val,C>T,ALFA,NP_055673.2:p.Ala602Val,NC_000001.11,205271242,C,T,14050:1
8,rs1375284370,NC_000001.11:205269051:C:T,TMCC2:9911,TMCC2,9911,NM_014858.4,NP_055673.2,MANE Select,1,205227946,...,+,NP_055673.2:p.Arg284Trp,C>T,ALFA,NP_055673.2:p.Arg284Trp,NC_000001.11,205269052,C,T,14050:1
9,rs1363717681,NC_000001.11:205271875:G:A,TMCC2:9911,TMCC2,9911,NM_014858.4,NP_055673.2,MANE Select,1,205227946,...,+,NP_055673.2:p.Val628Met,G>A,ALFA,NP_055673.2:p.Val628Met,NC_000001.11,205271876,G,A,14050:1
